In [1]:
from client import get_gvm_conn
import json

pyg = get_gvm_conn()

In [2]:
res = pyg.list_scanners()
print(res)
print(res[0]['name'])
print(res[0]['@id'])
print(res[1]['name'])
print(res[1]['@id'])

[{'owner': {'name': None}, 'name': 'CVE', 'comment': None, 'creation_time': '2024-07-05T13:21:44Z', 'modification_time': '2024-07-05T13:21:44Z', 'writable': '1', 'in_use': '0', 'permissions': {'permission': [{'name': 'get_scanners'}, {'name': 'get_scanners'}, {'name': 'get_scanners'}, {'name': 'get_scanners'}]}, 'host': None, 'port': '0', 'type': '3', 'ca_pub': None, 'credential': {'name': None, 'type': None, 'trash': '0', '@id': ''}, '@id': '6acd0832-df90-11e4-b9d5-28d24461215b'}, {'owner': {'name': None}, 'name': 'OpenVAS Default', 'comment': None, 'creation_time': '2024-07-05T13:21:44Z', 'modification_time': '2024-07-05T13:24:55Z', 'writable': '1', 'in_use': '1', 'permissions': {'permission': [{'name': 'get_scanners'}, {'name': 'get_scanners'}, {'name': 'get_scanners'}, {'name': 'get_scanners'}]}, 'host': '/run/ospd/ospd.sock', 'port': '0', 'type': '2', 'ca_pub': None, 'credential': {'name': None, 'type': None, 'trash': '0', '@id': ''}, '@id': '08b69003-5fc2-4037-a479-93b440211c73'}

In [ ]:
res = pyg.delete_scanner(scanner_id="a138b49a-5e0a-44d9-916b-8bceb8e8a282")
print(res)

### 创建目标

In [8]:
#33d0cd82-57c6-11e1-8ed1-406186ea4fc5
res = pyg.create_target('test_57', hosts=['223.193.36.146'], port_list_id='33d0cd82-57c6-11e1-8ed1-406186ea4fc5')
print(res['@id'])

f0e4b14c-8271-4e5d-96be-4ff21b29dae8


### 创建任务

In [10]:
tasks = []
for i in range(1):
# 创建任务
# '@id' : 'c11a8fbf-4403-4622-ac58-0ec85919bde9'
    time.sleep(1)
    res = pyg.create_task(name=f"test_for_146_for_{i}",target_id='f0e4b14c-8271-4e5d-96be-4ff21b29dae8', 
                          config_id='daba56c8-73ec-11df-a475-002264764cea', 
                          scanner_id='08b69003-5fc2-4037-a479-93b440211c73',
                            preferences={'assets_min_qod' : 30})
    tasks.append(res['@id'])
    print(f"task {res['@id']} append.")
    try:
        res = pyg.start_task(task_id=res['@id'])
        print(res)
    except:
        print(f"task {res['@id']} start failed.")

task d6127f5d-426d-4d66-ac59-556541c41b08 append.
{'report_id': 'f2f7799a-d4ce-4dbf-8564-a5baac281ed6', '@status': '202', '@status_text': 'OK, request submitted'}


In [4]:
res = pyg.list_tasks()
for re in res:
    print(re['@id'])

12cffbe5-6ed1-42df-b5bb-5f04120bf624
bc8332fb-c21a-4db9-af0a-f008bec0a301


In [3]:
tasks = pyg.list_tasks()
# for task in tasks:
#     try:
#         pyg.delete_task(task_id=task['@id'])
#         print(f"task {task['@id']} deleted.")
#     except:
#         print(f"task {res['@id']} deleted failed.")

### 获取任务进度
#### task的状态status包括Requested / Queued / Running / Done / Failed / Interapted
#### 可以通过 progress得到task进度

In [6]:
# tasks_d = pyg.list_tasks()
# tasks = []
# for task_d in tasks_d:
#     tasks.append(task_d['@id'])
print(time.asctime())
res = pyg.get_task(task_id=tasks[0])
print(res)
while True:
    for task in tasks[:]:
        res = pyg.get_task(task_id=task)
        print(f"task {task} progress is {res['progress']}")
        if(res['progress'] == '-1'):
            tasks.remove(task)
            print(f"task {task} finished.")
    if(len(tasks) == 0): break
    time.sleep(60)
print(time.asctime())

Fri Jan 19 09:19:13 2024
{'owner': {'name': 'admin'}, 'name': 'test_for_146_for_0', 'comment': None, 'creation_time': '2024-01-19T01:18:42Z', 'modification_time': '2024-01-19T01:18:42Z', 'writable': '1', 'in_use': '1', 'permissions': {'permission': {'name': 'Everything'}}, 'alterable': '0', 'usage_type': 'scan', 'config': {'name': 'Full and fast', 'trash': '0', '@id': 'daba56c8-73ec-11df-a475-002264764cea'}, 'target': {'name': 'test_57', 'trash': '0', '@id': 'd2416779-0232-4bac-9f8b-f61637e7a719'}, 'hosts_ordering': None, 'scanner': {'name': 'OpenVAS Default', 'type': '2', 'trash': '0', '@id': '08b69003-5fc2-4037-a479-93b440211c73'}, 'status': 'Running', 'progress': '0', 'report_count': {'finished': '0', '#text': '1'}, 'trend': None, 'schedule': {'name': None, 'trash': '0', '@id': ''}, 'schedule_periods': '0', 'current_report': {'report': {'timestamp': '2024-01-19T01:18:42Z', 'scan_start': '2024-01-19T01:19:01Z', 'scan_end': None, '@id': '60ee773e-5c34-48df-9142-27750d9bab76'}}, 'obser

### 获取report

In [9]:
# from utils import lxml_to_dict


res = pyg.list_reports(task_id='bc8332fb-c21a-4db9-af0a-f008bec0a301')
print(res[0]['@id'])
reportid = res[0]['@id']
# 获取report
# filter_str="apply_overrides=0 levels=hml rows=100 min_qod=30 first=1 sort-reverse=severity host~"223.193.36.146 result_hosts_only=1"
res = pyg.get_report(report_id=reportid, report_format_name='PDF', 
                     filter_str='apply_overrides=0 levels=hml rows=100 min_qod=70 first=1 sort-reverse=severity')
# res = lxml_to_dict(res)
# print(res['report'])

print(res)
# pretty_print(res)
# res = base64.b64decode(res)
# print(res)
# with open('./report1.pdf', 'wb+') as f:
#     f.write(res)

a9b43a8d-d125-4d50-856a-bdfda58ca5e2
Bytes
...


In [ ]:
res = pyg.list_report_formats()
for re in res:
    print(re)

In [14]:
print(len('4732ef12-42fd-4c4c-82d6-f044d6018021'))

36


## 根据结果翻译



## 获取任务结果

In [6]:
vuls = pyg.list_results(task_id="bc8332fb-c21a-4db9-af0a-f008bec0a301", filter_str="apply_overrides=0 levels=hml rows=100 min_qod=70 first=1 sort-reverse=severity")
print(len(vuls.data))
# print(json.dumps(res.data, indent=2))

79


## 统计数量获取html

In [8]:
if isinstance(vuls.data, list):
    print("list")
for vul in vuls.data:
    if("Spoofing" in vul["name"]):
       print(json.dumps(vul, indent=2)) 

list
{
  "name": "UnrealIRCd Authentication Spoofing Vulnerability",
  "owner": {
    "name": "admin"
  },
  "modification_time": "2024-09-17T12:22:53Z",
  "comment": null,
  "creation_time": "2024-09-17T12:22:53Z",
  "detection": {
    "result": {
      "details": {
        "detail": [
          {
            "name": "product",
            "value": "cpe:/a:unrealircd:unrealircd:3.2.8.1"
          },
          {
            "name": "location",
            "value": "6697/tcp"
          },
          {
            "name": "source_oid",
            "value": "1.3.6.1.4.1.25623.1.0.809884"
          },
          {
            "name": "source_name",
            "value": "UnrealIRCd Detection"
          }
        ]
      },
      "@id": "c70fec0e-3cba-4d4f-bebc-beb02ea5e4e4"
    }
  },
  "host": {
    "asset": {
      "@asset_id": "e3536d30-f918-40bf-a6e9-f16125d3ea4b"
    },
    "hostname": null,
    "#text": "192.168.220.132"
  },
  "port": "6697/tcp",
  "nvt": {
    "type": "nvt",
    "name

In [4]:
from datetime import datetime, timedelta
import re as regex
from sqlcve import CveSql
import html

sql = CveSql()

def format_str(value: str):
    value = html.escape(value)
    value = value.replace("\n \n", " <br /> ")
    value = value.replace("\n\n", " <br /> ")
    value = value.replace("\n", " <br /> ")
    return value

def get_vnl_en(vuln):
    vnl_en = ""
    if "detection" in vuln:
        detetction_str = " <b> Product Detection Result </b> <br /> "
        if "result" in vuln["detection"] and "details" in vuln["detection"]["result"] and "detail" in vuln["detection"]["result"]["details"]:
            detections = vuln["detection"]["result"]["details"]["detail"]
            for detection in detections:
                value = detection["value"]
                if isinstance(value, str):
                    value = format_str(value)
                    detetction_str +=  detection["name"] + ": " + value +   " <br /> "        
            vnl_en += detetction_str
    if "description" in vuln:
        detetction_str = " <b> Detection Result </b> <br /> "
        detectionstr = vuln["description"]
        if isinstance(detectionstr, str):   
            value = format_str(detectionstr)
            detetction_str += value + " <br /> "
            vnl_en += detetction_str
    if "nvt" in vuln and "tags" in vuln["nvt"]:
        tag_str = ""
        tags_str = vuln["nvt"]["tags"]
        tags = []
        if isinstance(tags_str, str):
            tags = tags_str.split("|")
        for tag in tags:
            kv = tag.split("=")
            value = format_str(kv[1])
            if value == ""  or value is None:
                continue
            if kv[0] in ["summary", "insight", "impact"]:
                tag_str += " <b> " + kv[0].capitalize() + " </b> " + " <br />" + value + " <br /> "
            elif kv[0] == "affected":
                tag_str += " <b> Affected Software/OS </b> " + " <br /> " + value + " <br /> "
            elif kv[0] == "vuldetect":
                tag_str += " <b> Detection Method </b> " + " <br /> " + value + " <br /> "
        vnl_en += tag_str
    return vnl_en
    
def trans_time(utc_time_str):
    utc_time = datetime.strptime(utc_time_str, "%Y-%m-%dT%H:%M:%SZ")
    beijing_offset = timedelta(hours=8)
    beijing_time = utc_time + beijing_offset
    return beijing_time.strftime('%Y-%m-%d %H:%M:%S')

def format_vulnerability(vuln):
    servity = "安全"
    severity_class = 'non-severity'
    if vuln['threat'] == 'High':
        severity_class = 'high-severity'
        servity = "高危"
    elif vuln['threat'] == 'Medium':
        severity_class = 'medium-severity'
        servity = "中危"
    elif vuln['threat'] == 'Low':
        severity_class = 'low-severity'
        servity = "低危"

    refstr = ""
    cve_id_list = []
    num = 0
    if "nvt" in vuln and "refs" in vuln["nvt"] and "ref" in vuln["nvt"]["refs"]:
        ref_list = vuln["nvt"]['refs']["ref"]
        if isinstance(ref_list, list):
            for ref in ref_list:
                if "@type" in ref and ref['@type'] == 'cve':
                    cve_id_list.append(ref['@id'])
                if num == 5:
                    break
                num += 1
                ref_str = format_str(ref['@type'] + ": " + ref['@id'])
                refstr += ref_str + " <br /> "
        elif isinstance(ref_list, dict):
            if "@type" in ref_list and ref_list['@type'] == 'cve':
                cve_id_list.append(ref_list['@id'])
            ref_str = format_str(ref_list['@type'] + ": " + ref_list['@id'])
            refstr += ref_str + " <br /> "
    
    title = vuln["name"]
    cvss = vuln["severity"]
    modification_time = trans_time(vuln["modification_time"])
    
    solution_en = ""
    if "nvt" in vuln and "solution" in vuln["nvt"]:
        if "@type" in vuln["nvt"]["solution"]:
            solution_en = vuln["nvt"]["solution"]["@type"] + ": "
        if "#text" in vuln["nvt"]["solution"]:
            solution_en += vuln["nvt"]["solution"]["#text"]
    solution_en = format_str(solution_en)
    solution = solution_en
    
    # 获取中文漏洞信息
    zh = False
    data = None
    # qod = vuln["qod"]["value"]
    port = vuln["port"]
    # 整合英文信息
    description_en = get_vnl_en(vuln)
    description = description_en
    
    if cve_id_list:
        for cve_id in cve_id_list:
            zh, data = sql.find_vul(cve_id)
            if zh:
                break
    if zh:
        if data[8] is not None:
            title = data[8]
        if data[9] is not None:
            solution = data[9] + " <br /> " + solution_en
        if data[11] is not None:
            description = data[11] + " <br /> " + description_en
    
    return f"""
    <div>
        <h3>{title}</h3>
        <table>
            <tr class="{severity_class}">
                <td style="width:10%">严重性</td>
                <td style="width:90%">{servity}</td>
            </tr>
            <tr>
                <td style="width:10%">发现时间</td>
                <td style="width:90%">{modification_time}</td>
            </tr>
            <tr>
                <td style="width:10%">漏洞描述</td>
                <td style="width:90%">{description}</td>
            </tr>
            <tr>
                <td style="width:10%">CVSS评分</td>
                <td style="width:90%">{cvss}</td>
            </tr>
            <tr>
                <td style="width:10%">扫描端口</td>
                <td style="width:90%">{port}</td>
            </tr>
            <tr>
                <td style="width:10%">建议措施</td>
                <td style="width:90%">{solution}</td>
            </tr>
            <tr>
                <td style="width:10%">参考资料</td>
                <td style="width:90%">{refstr}</td>
            </tr>
        </table>
    </div>
    """

In [ ]:
res = pyg.list_configs()
print(json.dumps(res.data, indent=2))

In [4]:
config = pyg.get_config("daba56c8-73ec-11df-a475-002264764cea")
# print(json.dumps(config.data, indent=2))
# print( isinstance(config.data, dict))
# print(len(config.data))

for k in config.data.keys():
    print(k)

print( isinstance(config.data["families"], dict))
for k in config.data["families"].keys():
    print(k)
print( isinstance(config.data["families"]["family"], list))
print( len(config.data["families"]["family"]))
for k in config.data["families"]["family"][1].keys():
    print(k)

print(config.data["families"]["family"][1]["name"])
print(config.data["families"]["family"][1]["nvt_count"])
print(config.data["families"]["family"][1]["growing"])

owner
name
comment
creation_time
modification_time
writable
in_use
permissions
family_count
nvt_count
type
usage_type
predefined
families
max_nvt_count
known_nvt_count
preferences
nvt_selectors
@id
True
family
True
58
name
nvt_count
max_nvt_count
growing
Amazon Linux Local Security Checks
748
1


In [ ]:
family = "Amazon Linux Local Security Checks"
config_str = "daba56c8-73ec-11df-a475-002264764cea"
res = pyg.list_config_nvts(details=False, config_id=config_str, family=family)
print(json.dumps(res.data, indent=2))